# Tie contact 3d

Author(s): Jukka Aho

## Solid block

Geometry and mesh:

<div class="main_block" style="max-width: 800px;">
  <div class="inner_block" style="display: inline-block; float:left; width:50%;">
    <img src="http://results.juliafem.org/2015-12-13-box_geom.png"
         style="width:100%; height:auto; vertical-align:middle;">
  </div>
  <div class="inner_block" style="display: inline-block; float:left; width:50%;">
    <img src="http://results.juliafem.org/2015-12-13-box_bcs.png"
         style="width:100%; height:auto; vertical-align:middle;">
  </div>
</div>

In [1]:
using JuliaFEM
using JuliaFEM.Preprocess: parse_aster_med_file
using JuliaFEM.Core: LinearElasticityProblem, DirichletProblem, get_connectivity, Quad4, Hex8, DirectSolver

In [2]:
mesh = parse_aster_med_file(Pkg.dir("JuliaFEM")*"/geometry/unit_box/mesh.med")
#mesh = parse_aster_med_file(Pkg.dir("JuliaFEM")*"/geometry/unit_box/twoelem_box.med")

INFO: Found 4 element sets: SYM23, SYM12, SYM13, LOAD


Dict{ASCIIString,Any} with 2 entries:
  "nodes"        => Dict(2=>[0.0,0.0,0.0],11=>[0.0,0.3333333333333333,1.0],39=>…
  "connectivity" => Dict(68=>(:QU4,:OTHER,[45,47,48,46]),2=>(:SE2,:OTHER,[9,10]…

In [5]:
# interior elements are of type HE8
field_problem = LinearElasticityProblem()
for (elid, (eltype, elset, elcon)) in mesh["connectivity"]
    eltype == :HE8 || continue
    element = Hex8(elcon)
    element["geometry"] = Vector{Float64}[mesh["nodes"][i] for i in get_connectivity(element)]
    element["youngs modulus"] = 900.0
    element["poissons ratio"] = 0.25
    push!(field_problem, element)
end
# Neumann boundary condition, traction force -100 on Z direction for element set LOAD
for (elid, (eltype, elset, elcon)) in mesh["connectivity"]
    (eltype == :QU4) && (elset == :LOAD) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mesh["nodes"][i] for i in get_connectivity(element)]
    element["displacement traction force"] = Vector{Float64}[[0.0, 0.0, -100.0] for i=1:4]
    push!(field_problem, element)
end
info("created $(length(field_problem.elements)) elements.")

INFO: created 36 elements.


In [6]:
# boundary conditions
boundary_problem = DirichletProblem("displacement", 3)
for (elid, (eltype, elset, elcon)) in mesh["connectivity"]
    (eltype == :QU4) && (elset in [:SYM23, :SYM12, :SYM13]) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mesh["nodes"][i] for i in get_connectivity(element)]
    if elset == :SYM23
        element["displacement 1"] = 0.0
    elseif elset == :SYM12
        element["displacement 3"] = 0.0
    elseif elset == :SYM13
        element["displacement 2"] = 0.0
    end
    push!(boundary_problem, element)
end
info("created $(length(boundary_problem.elements)) boundary elements.")

INFO: created 27 boundary elements.


In [7]:
solver = DirectSolver()
solver.name = "block"
solver.nonlinear_problem = false
#solver.method = :UMFPACK
solver.dump_matrices = true
push!(solver, field_problem)
push!(solver, boundary_problem)
call(solver, 0.0)

INFO: # of field problems: 1
INFO: # of boundary problems: 1
INFO: Starting iteration 1
INFO: Assembling field problems...
INFO: Assembling body 1...
INFO: dim = 192
INFO: Assembling boundary problems...
INFO: Assembling boundary 1...
INFO: dumping matrices to disk, file = matrices_block_host_1_iteration_1.jld
INFO: Solving system
INFO: CHOLMOD: all dofs = 192
INFO: CHOLMOD: interior dofs = 144
INFO: CHOLMOD: boundary dofs = 48
INFO: CHOLMOD: displacement on boundary solved.
INFO: CHOLMOD: homogeneous dirichlet boundary
INFO: CHOLMOD: LDLt factorization done in 0.0012049674987792969 seconds
INFO: CHOLMOD: solved in 0.05479288101196289 seconds. norm = 0.5879447357921324
INFO: timing info for iteration:
INFO: boundary assembly       : 0.09608697891235352


(1,true)

INFO: field assembly          : 1.0157549381256104
INFO: dump matrices to disk   : 0.9569120407104492
INFO: solve problem           : 0.48387885093688965
INFO: update element data     : 0.01578807830810547
INFO: non-linear iteration    : 2.568441867828369
INFO: solver finished in 2.7200798988342285 seconds.


In [8]:
nid = 0
for (nid, coords) in mesh["nodes"]
    if isapprox(coords, [1.0, 1.0, 1.0])
        info("nid near corner = $nid")
        break
    end
end
nid

INFO: nid near corner = 7


7

In [9]:
using JuliaFEM.Test
known_value = [1/36, 1/36, -1/9]
for element in field_problem.elements
    i = indexin([nid], get_connectivity(element))[1]
    i != 0 || continue
    X = element("geometry", 0.0)
    u = element("displacement", 0.0)
    info("displacement X = $(X[i]), u = $(u[i])")
    @test isapprox(u[i], known_value)
end

INFO: displacement X = [1.0,1.0,1.0], u = [0.027777777777777794,0.02777777777777784,-0.11111111111111129]
INFO: displacement X = [1.0,1.0,1.0], u = [0.027777777777777794,0.02777777777777784,-0.11111111111111129]


In [10]:
xdoc, xmodel = JuliaFEM.Postprocess.xdmf_new_model()
coll = JuliaFEM.Postprocess.xdmf_new_temporal_collection(xmodel)
grid = JuliaFEM.Postprocess.xdmf_new_grid(coll; time=0.0)

Xg = Dict{Int64, Vector{Float64}}()
ug = Dict{Int64, Vector{Float64}}()
for element in field_problem.elements
    conn = get_connectivity(element)
    X = element("geometry", 0.0)
    u = element("displacement", 0.0)
    for (i, c) in enumerate(conn)
        Xg[c] = X[i]
        ug[c] = u[i]
    end
end
perm = sort(collect(keys(Xg)))
nodes = Vector{Float64}[Xg[i] for i in perm]
disp = Vector{Float64}[ug[i] for i in perm]
elements = []
for el in field_problem.elements
    isa(el, JuliaFEM.Core.Element{JuliaFEM.Core.Hex8}) || continue
    push!(elements, (:Hex8, get_connectivity(el)))
end
#elements
JuliaFEM.Postprocess.xdmf_new_mesh!(grid, nodes, elements)
JuliaFEM.Postprocess.xdmf_new_nodal_field!(grid, "displacement", disp)
JuliaFEM.Postprocess.xdmf_save_model(xdoc, "/tmp/foobar2.xmf");

INFO: XDFM: ndim = 192


<img src="http://results.juliafem.org/2015-12-13-box_results.png" width=300px style="float:left">

## Block divived to two parts

Block is now divided to two parts and meshes are tied using mortar method.

<div class="main_block" style="max-width: 800px;">
  <div class="inner_block" style="display: inline-block; float:left; width:50%;">
    <img src="http://results.juliafem.org/2015-12-13-divided-block-geometry.png"
         style="width:100%; height:auto; vertical-align:middle;">
  </div>
  <div class="inner_block" style="display: inline-block; float:left; width:50%;">
    <img src="http://results.juliafem.org/2015-12-13-divided-block-both-parts.png"
         style="width:100%; height:auto; vertical-align:middle;">
  </div>
</div>

In [1]:
using JuliaFEM
using JuliaFEM.Preprocess: parse_aster_med_file
using JuliaFEM.Core: LinearElasticityProblem, DirichletProblem, get_connectivity, Quad4, Hex8, LinearSolver
mesh_lower = parse_aster_med_file(Pkg.dir("JuliaFEM")*"/geometry/unit_box/LOWER_BLOCK.med")
mesh_upper = parse_aster_med_file(Pkg.dir("JuliaFEM")*"/geometry/unit_box/UPPER_BLOCK.med")

INFO: Found 4 element sets: LSYM23, LSYM12, LSYM13, LOWER_BLOCK_TO_UPPER_BLOCK
INFO: Found 4 element sets: USYM23, UPPER_BLOCK_TO_LOWER_BLOCK, USYM13, LOAD


Dict{ASCIIString,Any} with 2 entries:
  "nodes"        => Dict(2=>[0.0,0.0,1.0],11=>[0.0,0.6666666666666666,1.0],39=>…
  "connectivity" => Dict(68=>(:QU4,:UPPER_BLOCK_TO_LOWER_BLOCK,[26,4,14,42]),2=…

In [2]:
sort(collect(keys(mesh_lower["nodes"])))'

1x75 Array{Int64,2}:
 1  2  3  4  5  6  7  8  9  10  11  12  …  67  68  69  70  71  72  73  74  75

In [3]:
sort(collect(keys(mesh_upper["nodes"])))'

1x48 Array{Int64,2}:
 1  2  3  4  5  6  7  8  9  10  11  12  …  40  41  42  43  44  45  46  47  48

Need to renumber nodes.

In [4]:
new_node_numbering = Dict{Int64, Int64}()
upper_node_ids = sort(collect(keys(mesh_upper["nodes"])))
for (k, old_node_id) in enumerate(upper_node_ids)
    new_node_numbering[old_node_id] = 75+k
end
new_node_numbering

Dict{Int64,Int64} with 48 entries:
  2  => 77
  11 => 86
  39 => 114
  46 => 121
  25 => 100
  42 => 117
  29 => 104
  8  => 83
  20 => 95
  14 => 89
  31 => 106
  33 => 108
  18 => 93
  26 => 101
  35 => 110
  17 => 92
  44 => 119
  4  => 79
  37 => 112
  45 => 120
  13 => 88
  30 => 105
  1  => 76
  47 => 122
  32 => 107
  ⋮  => ⋮

In [5]:
function renumber!(mesh, node_numbering)
    old_nodes = mesh["nodes"]
    new_nodes = typeof(old_nodes)()
    for (node_id, node_coords) in old_nodes
        new_node_id = node_numbering[node_id]
        new_nodes[new_node_id] = node_coords
    end
    mesh["nodes"] = new_nodes
    for (elid, (eltype, elset, elcon)) in mesh["connectivity"]
        new_elcon = [node_numbering[node_id] for node_id in elcon]
        mesh["connectivity"][elid] = (eltype, elset, new_elcon)
    end
    return mesh
end

renumber!(mesh_upper, new_node_numbering)
sort(collect(keys(mesh_upper["nodes"])))'

1x48 Array{Int64,2}:
 76  77  78  79  80  81  82  83  84  85  …  117  118  119  120  121  122  123

In [6]:
mesh_upper["connectivity"]

Dict{Int64,Tuple{Symbol,Symbol,Array{Int64,1}}} with 92 entries:
  68 => (:QU4,:UPPER_BLOCK_TO_LOWER_BLOCK,[101,79,89,117])
  2  => (:SE2,:OTHER,[77,84])
  89 => (:HE8,:OTHER,[114,102,103,115,121,118,119,123])
  11 => (:SE2,:OTHER,[90,80])
  39 => (:QU4,:USYM13,[93,106,90,80])
  46 => (:QU4,:LOAD,[96,109,108,95])
  85 => (:HE8,:OTHER,[94,93,114,116,107,106,121,120])
  25 => (:SE2,:OTHER,[99,83])
  55 => (:QU4,:OTHER,[95,113,112,96])
  42 => (:QU4,:USYM13,[106,92,81,90])
  66 => (:QU4,:UPPER_BLOCK_TO_LOWER_BLOCK,[116,88,76,94])
  58 => (:QU4,:OTHER,[113,101,100,112])
  29 => (:SE2,:OTHER,[79,101])
  59 => (:QU4,:OTHER,[87,79,101,113])
  8  => (:SE2,:OTHER,[88,76])
  74 => (:QU4,:OTHER,[99,83,103,119])
  90 => (:HE8,:OTHER,[89,117,101,79,105,122,113,87])
  57 => (:QU4,:OTHER,[112,100,83,99])
  20 => (:SE2,:OTHER,[96,95])
  78 => (:HE8,:OTHER,[104,120,122,105,85,110,108,86])
  14 => (:SE2,:OTHER,[92,91])
  31 => (:SE2,:OTHER,[103,102])
  70 => (:QU4,:OTHER,[98,119,118,97])
  33 => (:QU4,:

In [7]:
# interior elements are of type HE8
field_problem = LinearElasticityProblem()

for (elid, (eltype, elset, elcon)) in mesh_lower["connectivity"]
    eltype == :HE8 || continue
    element = Hex8(elcon)
    element["geometry"] = Vector{Float64}[mesh_lower["nodes"][i] for i in get_connectivity(element)]
    element["youngs modulus"] = 900.0
    element["poissons ratio"] = 0.25
    #info("lower: add element with connectivity $elcon")
    push!(field_problem, element)
end

for (elid, (eltype, elset, elcon)) in mesh_upper["connectivity"]
    eltype == :HE8 || continue
    element = Hex8(elcon)
    element["geometry"] = Vector{Float64}[mesh_upper["nodes"][i] for i in get_connectivity(element)]
    element["youngs modulus"] = 900.0
    element["poissons ratio"] = 0.25
    #info("upper: add element with connectivity $elcon")
    push!(field_problem, element)
end

# Neumann boundary condition, traction force -100 on Z direction for element set LOAD
for (elid, (eltype, elset, elcon)) in mesh_upper["connectivity"]
    (eltype == :QU4) && (elset == :LOAD) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mesh_upper["nodes"][i] for i in get_connectivity(element)]
    element["displacement traction force"] = Vector{Float64}[[0.0, 0.0, -100.0] for i=1:4]
    push!(field_problem, element)
end
info("created $(length(field_problem.elements)) elements.")

INFO: created 59 elements.


In [8]:
# boundary conditions
boundary_problem = DirichletProblem("displacement", 3)

for (elid, (eltype, elset, elcon)) in mesh_lower["connectivity"]
    (eltype == :QU4) && (elset in [:LSYM23, :LSYM12, :LSYM13]) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mesh_lower["nodes"][i] for i in get_connectivity(element)]
    if elset == :LSYM23
        element["displacement 1"] = 0.0
    elseif elset == :LSYM12
        element["displacement 3"] = 0.0
    elseif elset == :LSYM13
        element["displacement 2"] = 0.0
    end
    push!(boundary_problem, element)
end

for (elid, (eltype, elset, elcon)) in mesh_upper["connectivity"]
    (eltype == :QU4) && (elset in [:USYM23, :USYM12, :USYM13]) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mesh_upper["nodes"][i] for i in get_connectivity(element)]
    if elset == :USYM23
        element["displacement 1"] = 0.0
    elseif elset == :USYM12
        element["displacement 3"] = 0.0
    elseif elset == :USYM13
        element["displacement 2"] = 0.0
    end
    push!(boundary_problem, element)
end

info("created $(length(boundary_problem.elements)) boundary elements.")

INFO: created 44 boundary elements.


### Creating tie contact
- define slave element surface (the one where integration happend)
- define potential master elements for slave elements

In [9]:
using JuliaFEM.Core: Element, MortarProblem, calculate_normal_tangential_coordinates!

mortar_side = mesh_upper
nonmortar_side = mesh_lower

master_elements = JuliaFEM.Core.Element[]
for (elid, (eltype, elset, elcon)) in mortar_side["connectivity"]
    (eltype == :QU4) && (elset == :UPPER_BLOCK_TO_LOWER_BLOCK) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[mortar_side["nodes"][i] for i in get_connectivity(element)]
    push!(master_elements, element)
end

contact_problem = MortarProblem("displacement", 3)
for (elid, (eltype, elset, elcon)) in nonmortar_side["connectivity"]
    (eltype == :QU4) && (elset == :LOWER_BLOCK_TO_UPPER_BLOCK) || continue
    element = Quad4(elcon)
    element["geometry"] = Vector{Float64}[nonmortar_side["nodes"][i] for i in get_connectivity(element)]
    element["master elements"] = master_elements
    calculate_normal_tangential_coordinates!(element, 0.0)
    push!(contact_problem, element)
end

In [10]:
using JuliaFEM.Core: DirectSolver
solver = DirectSolver()
solver.name = "tie_contact_3d"
solver.method = :UMFPACK
solver.nonlinear_problem = false
solver.max_iterations = 1
solver.dump_matrices = true
push!(solver, field_problem)
push!(solver, boundary_problem)
push!(solver, contact_problem)
call(solver, 0.0)

INFO: # of field problems: 1
INFO: # of boundary problems: 2
INFO: Starting iteration 1
INFO: Assembling field problems...
INFO: Assembling body 1...
INFO: dim = 369
INFO: Assembling boundary problems...
INFO: Assembling boundary 1...
INFO: Assembling boundary 2...
INFO: dumping matrices to disk, file = matrices_tie_contact_3d_host_1_iteration_1.jld
INFO: Solving system
INFO: UMFPACK: solved in 0.18637418746948242 seconds. norm = 0.7276121358181449
INFO: timing info for iteration:


(1,true)

INFO: boundary assembly       : 1.6767330169677734
INFO: field assembly          : 1.1170330047607422
INFO: dump matrices to disk   : 1.0959408283233643
INFO: solve problem           : 0.3340458869934082
INFO: update element data     : 0.022793054580688477
INFO: non-linear iteration    : 4.246567964553833
INFO: solver finished in 4.391422986984253 seconds.


In [11]:
using JuliaFEM.Test

nid = 0
for (nid, coords) in mesh_upper["nodes"]
    if isapprox(coords, [1.0, 1.0, 1.0])
        info("nid near corner = $nid")
        break
    end
end
nid

known_value = [1/36, 1/36, -1/9]

for element in field_problem.elements
    i = indexin([nid], get_connectivity(element))[1]
    i != 0 || continue
    X = element("geometry", 0.0)
    u = element("displacement", 0.0)
    info("displacement X = $(X[i]), u = $(u[i])")
    @test isapprox(-u[i], known_value)
end

INFO: nid near corner = 82
INFO: displacement X = [1.0,1.0,1.0], u = [0.02865803993668297,0.02865803993668297,-0.11229776250350555]


LoadError: LoadError: test failed: isapprox(-(u[i]),known_value)
 in expression: isapprox(-(u[i]),known_value)
while loading In[11], in expression starting on line 14

In [28]:
xdoc, xmodel = JuliaFEM.Postprocess.xdmf_new_model()
coll = JuliaFEM.Postprocess.xdmf_new_temporal_collection(xmodel)
grid = JuliaFEM.Postprocess.xdmf_new_grid(coll; time=0.0)

Xg = Dict{Int64, Vector{Float64}}()
ug = Dict{Int64, Vector{Float64}}()
for element in field_problem.elements
    conn = get_connectivity(element)
    X = element("geometry", 0.0)
    u = element("displacement", 0.0)
    for (i, c) in enumerate(conn)
        Xg[c] = X[i]
        ug[c] = u[i]
    end
end
perm = sort(collect(keys(Xg)))
nodes = Vector{Float64}[Xg[i] for i in perm]
disp = Vector{Float64}[ug[i] for i in perm]
elements = []
for el in field_problem.elements
    isa(el, JuliaFEM.Core.Element{JuliaFEM.Core.Hex8}) || continue
    push!(elements, (:Hex8, get_connectivity(el)))
end
#elements
JuliaFEM.Postprocess.xdmf_new_mesh!(grid, nodes, elements)
JuliaFEM.Postprocess.xdmf_new_nodal_field!(grid, "displacement", disp)
JuliaFEM.Postprocess.xdmf_save_model(xdoc, "/tmp/foobar3.xmf");

INFO: XDFM: ndim = 369


<img src="http://results.juliafem.org/2015-12-14-divided-block-results.png"
         style="width:30%; height:auto; vertical-align:middle;">